In [1]:
from retnet_exp.modeling_retnet import RetNetForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
import matplotlib.pyplot as plt

/home/k_ishikawa/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = "Spiral-AI/Spiral-RetNet-3b-base"
model_retnet = RetNetForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
)
tok_retnet = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.92s/it]


In [4]:
def precompute_kv_retnet(model, inputs):
    with torch.inference_mode():
        start = time.time()
        outputs = model(
            **inputs, return_dict=True, use_cache=True, forward_impl="parallel"
        )
        end = time.time()
        elapsed = end - start
        # print(f"Precompute: {elapsed} seconds")
    return outputs, elapsed

In [5]:
def generate_retnet(
    model, input_ids, precomputed, mode: str, max_new_tokens=50, **kwargs
):
    input_ids = input_ids.to(model.device)
    with torch.inference_mode():
        history = input_ids
        start = time.time()
        if mode == "parallel":
            for _ in range(max_new_tokens):
                outputs = model(
                    input_ids=history,
                    past_key_values=None,
                    return_dict=True,
                    forward_impl="parallel",
                    **kwargs,
                )
                logit = outputs.logits[:, -1, :]  # [batch_size, vocab_size]
                generated = torch.argmax(logit, dim=-1, keepdim=True)
                history = torch.cat([history, generated], dim=-1)

            end = time.time()
            elapsed = end - start
            print(f"Parallel Inference: {elapsed} seconds")
            return history, elapsed

        elif mode == "recurrent":
            past_kv = precomputed.past_key_values
            for _ in range(max_new_tokens):
                outputs = model(
                    input_ids=history,
                    retention_mask=None,
                    forward_impl="recurrent",
                    past_key_values=past_kv,
                    use_cache=True,
                    return_dict=True,
                )
                logit = outputs.logits[:, -1, :]  # [batch_size, vocab_size]
                past_kv = outputs.past_key_values
                generated = torch.argmax(logit, dim=-1, keepdim=True)
                history = torch.cat([history, generated], dim=-1)
            end = time.time()
            elapsed = end - start
            print(f"Reccurent Inference: {elapsed} seconds")
            return history, elapsed

In [6]:
def inference_retnet(model, tokenizer, text, mode: str, max_new_tokens=50):
    print()
    print("=" * 80)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    print("# of input tokens:", len(inputs["input_ids"][0]), "tokens")
    precomputed, precompute_time = precompute_kv_retnet(model, inputs)
    outputs, elapsed = generate_retnet(
        model, inputs["input_ids"], precomputed, mode, max_new_tokens=max_new_tokens
    )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(text) :]
    print("Input:", text)
    print("Output:", generated)
    wps = len(generated) / elapsed
    print(f"Throughput ({mode}): {wps} wps")
    return generated, wps

In [7]:
texts = [
    """私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。今日読んだニュースは以下の通りです。「米国の中央銀行にあたる米連邦準備制度理事会（FRB）は、1日に開いた米連邦公開市場委員会（FOMC）で、6会合連続で政策金利を据え置くと決めた。現状維持は市場の予想通りで、FRBのパウエル議長は記者会見で、高金利を長く保つ意向を明らかにした一方で、再び利上げする可能性は「低いと思う」と述べた。
パウエル氏が利上げの可能性は低いと言及したことで、米金利の上昇による日米の金利差拡大への観測が後退。ニューヨーク市場で、円相場はやや円高ドル安方向に振れ、米東部時間1日午後3時半（日本時間2日午前4時半）ごろには1ドル=157円台半ばで取引されていた。」そういえば、繰り返しになりますが、私が嫌いなスポーツは""",
    """私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。そういえば、繰り返しになりますが、私が嫌いなスポーツは""",
    "私が嫌いなスポーツはカレーです。私が嫌いなスポーツは",
    """私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。今日読んだニュースは以下の通りです。「【ワシントン＝大内清】バイデン米政権は１日、ウクライナ侵略を続けるロシア軍が、化学兵器禁止条約（ＣＷＣ）で禁じられている化合物クロロピクリンをウクライナ軍に対して戦場で使用したと断定した。バイデン政権は同日、化学兵器の製造に関与した露政府機関などを含め、軍需産業などの計約２８０団体・個人に制裁を科すと発表した。

【写真】ロシア軍は民間人を巻き込む残虐な攻撃を繰り返している

ウクライナのゼレンスキー政権は、露軍がクロロピクリンを含む複数の化学兵器を戦場で使用していると主張していた。

クロロピクリンは暴徒鎮圧用の催涙ガスにも用いられる化合物で、第一次世界大戦では窒息性の毒ガス兵器としても使用された。国務省は「要塞化された陣地からウクライナ軍を追い払うこと」などを目的に使用されたと指摘。化学兵器計画に関連する露政府の３機関と露企業４社を制裁対象に指定した。

またバイデン政権は、ロシアの北極圏での液化天然ガス（ＬＮＧ）事業や、露国営原子力企業「ロスアトム」の関連会社、軍需産業などに従事する企業・個人らを幅広く制裁対象に追加。２月に露反体制派指導者ナワリヌイ氏が獄中死したことに絡み、露刑務当局者ら３人にも在米資産凍結などの制裁を発動した。」そういえば、繰り返しになりますが、私が嫌いなスポーツは""",
    """私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。今日読んだニュースは以下の通りです。母乳に含まれる抗体が子どもの脳に影響を与える仕組みを、群馬大大学院の定方哲史准教授（４９）と長男で同大医学部３年の瑞樹さん（２０）らの研究グループが解明した。抗体と結合した脳内の細胞が神経細胞の生存に関係するたんぱく質を分泌することを突き止めた。研究成果は国際科学誌「ジャーナル・オブ・ニューロインフラメーション」に発表した。【庄司哲也】

【簡単に解説】母乳が子どもの脳に影響を与える仕組み

　これまでＩｇＧと呼ばれる抗体が母乳から子どもの血液に移送され新生児の免疫力を高めることは分かっていたが、脳への影響は明らかになっていなかった。

　定方准教授らのグループは、実験用の子どものマウスの脳内でＩｇＧが免疫細胞ミクログリアと結合していることに着目。結合したミクログリアが、脳を構成する神経細胞のニューロンの生存に関係するたんぱく質の１型インターフェロンを分泌していることを解明した。

　さらに母親からＩｇＧを受け取るために必要なＦｃＲｎと呼ばれる受容体を欠失させたマウスを調べたところ、ミクログリアや特定のニューロンなど脳内細胞が減少していることも分かった。

　ただ、人間では授乳期に母親が感染症にかかると、ＩｇＧが増えた母乳が、子どもの脳に悪影響を与えてしまう可能性があり、母乳から移送されるＩｇＧの量がどのような影響を与えるかは不明という。

　定方准教授は「ＩｇＧを受け取れないマウスの観察では、社会性行動に有意な変化が見られた。人の母親のＩｇＧが子の脳に良い影響を与えるのか、悪い影響となるのかはさらに研究が必要で、今後明らかにしていきたい」と話している。母乳に含まれる抗体が子どもの脳に影響を与える仕組みを、群馬大大学院の定方哲史准教授（４９）と長男で同大医学部３年の瑞樹さん（２０）らの研究グループが解明した。抗体と結合した脳内の細胞が神経細胞の生存に関係するたんぱく質を分泌することを突き止めた。研究成果は国際科学誌「ジャーナル・オブ・ニューロインフラメーション」に発表した。【庄司哲也】

【簡単に解説】母乳が子どもの脳に影響を与える仕組み

　これまでＩｇＧと呼ばれる抗体が母乳から子どもの血液に移送され新生児の免疫力を高めることは分かっていたが、脳への影響は明らかになっていなかった。

　定方准教授らのグループは、実験用の子どものマウスの脳内でＩｇＧが免疫細胞ミクログリアと結合していることに着目。結合したミクログリアが、脳を構成する神経細胞のニューロンの生存に関係するたんぱく質の１型インターフェロンを分泌していることを解明した。

　さらに母親からＩｇＧを受け取るために必要なＦｃＲｎと呼ばれる受容体を欠失させたマウスを調べたところ、ミクログリアや特定のニューロンなど脳内細胞が減少していることも分かった。

　ただ、人間では授乳期に母親が感染症にかかると、ＩｇＧが増えた母乳が、子どもの脳に悪影響を与えてしまう可能性があり、母乳から移送されるＩｇＧの量がどのような影響を与えるかは不明という。

　定方准教授は「ＩｇＧを受け取れないマウスの観察では、社会性行動に有意な変化が見られた。人の母親のＩｇＧが子の脳に良い影響を与えるのか、悪い影響となるのかはさらに研究が必要で、今後明らかにしていきたい」と話している。母乳に含まれる抗体が子どもの脳に影響を与える仕組みを、群馬大大学院の定方哲史准教授（４９）と長男で同大医学部３年の瑞樹さん（２０）らの研究グループが解明した。抗体と結合した脳内の細胞が神経細胞の生存に関係するたんぱく質を分泌することを突き止めた。研究成果は国際科学誌「ジャーナル・オブ・ニューロインフラメーション」に発表した。【庄司哲也】

【簡単に解説】母乳が子どもの脳に影響を与える仕組み

　これまでＩｇＧと呼ばれる抗体が母乳から子どもの血液に移送され新生児の免疫力を高めることは分かっていたが、脳への影響は明らかになっていなかった。

　定方准教授らのグループは、実験用の子どものマウスの脳内でＩｇＧが免疫細胞ミクログリアと結合していることに着目。結合したミクログリアが、脳を構成する神経細胞のニューロンの生存に関係するたんぱく質の１型インターフェロンを分泌していることを解明した。

　さらに母親からＩｇＧを受け取るために必要なＦｃＲｎと呼ばれる受容体を欠失させたマウスを調べたところ、ミクログリアや特定のニューロンなど脳内細胞が減少していることも分かった。

　ただ、人間では授乳期に母親が感染症にかかると、ＩｇＧが増えた母乳が、子どもの脳に悪影響を与えてしまう可能性があり、母乳から移送されるＩｇＧの量がどのような影響を与えるかは不明という。

　定方准教授は「ＩｇＧを受け取れないマウスの観察では、社会性行動に有意な変化が見られた。人の母親のＩｇＧが子の脳に良い影響を与えるのか、悪い影響となるのかはさらに研究が必要で、今後明らかにしていきたい」と話している。そういえば、繰り返しになりますが、私が嫌いなスポーツは""",
    """私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。今日読んだニュースは以下の通りです。母乳に含まれる抗体が子どもの脳に影響を与える仕組みを、群馬大大学院の定方哲史准教授（４９）と長男で同大医学部３年の瑞樹さん（２０）らの研究グループが解明した。抗体と結合した脳内の細胞が神経細胞の生存に関係するたんぱく質を分泌することを突き止めた。研究成果は国際科学誌「ジャーナル・オブ・ニューロインフラメーション」に発表した。【庄司哲也】

【簡単に解説】母乳が子どもの脳に影響を与える仕組み

　これまでＩｇＧと呼ばれる抗体が母乳から子どもの血液に移送され新生児の免疫力を高めることは分かっていたが、脳への影響は明らかになっていなかった。

　定方准教授らのグループは、実験用の子どものマウスの脳内でＩｇＧが免疫細胞ミクログリアと結合していることに着目。結合したミクログリアが、脳を構成する神経細胞のニューロンの生存に関係するたんぱく質の１型インターフェロンを分泌していることを解明した。

　さらに母親からＩｇＧを受け取るために必要なＦｃＲｎと呼ばれる受容体を欠失させたマウスを調べたところ、ミクログリアや特定のニューロンなど脳内細胞が減少していることも分かった。

　ただ、人間では授乳期に母親が感染症にかかると、ＩｇＧが増えた母乳が、子どもの脳に悪影響を与えてしまう可能性があり、母乳から移送されるＩｇＧの量がどのような影響を与えるかは不明という。

　定方准教授は「ＩｇＧを受け取れないマウスの観察では、社会性行動に有意な変化が見られた。人の母親のＩｇＧが子の脳に良い影響を与えるのか、悪い影響となるのかはさらに研究が必要で、今後明らかにしていきたい」と話している。看護学校生が卒業後に系列病院で一定期間働けば返済が免除される奨学金について、その病院で勤務できない場合、返済義務があるかが大阪地裁の民事訴訟で争われている。学校の運営法人が卒業生３人に返済を求め、卒業生側は法人の都合で不採用にしたのに返済を求めるのは信義則違反だとする。同様の奨学金は看護業界で普及しており、トラブルは後を絶たない。（岸田藍）

　法人は大阪府内の社会医療法人で、精神科病院に併設する看護専門学校（２年課程）を運営している。訴訟記録などによると、学生は入学前の書類審査や面接を経て奨学金を受給でき、卒業後に法人の職員として２年以上勤務すれば返済が全額免除される。

　３人は２０１９年４月に入学した男性１人（２７）、女性２人（３６歳と４４歳）。経済的な理由で、学費名目の奨学金１７８万円をそれぞれ受給した。受給者は同学年約３０人のうち半数ほどだった。

　３人は看護師資格を取得し、卒業後は法人の病院での勤務を希望したが、採用試験で２０年７月に不採用とされ、別の病院に就職した。他の受給者の大半は採用された。

　法人側は「生活状況を踏まえ、奨学金の返済額をそれぞれ１００万円に減らす」と３人に提案したが、合意に至らず、１７８万円ずつの返済を求め、昨年８月に提訴した。

　法人側は、奨学金の貸与が法人職員としての採用を約束するものではないと主張。採用試験で実施した面接や心理テストで、合格水準に達しなかったとし、返済義務があると強調する。

　卒業生側が重視するのは、入学前に配布された奨学金の募集案内の内容だ。受給の条件として「法人の病院に就職する者」を挙げ、貸与取り消しについては「法人への就職を希望しなくなった場合」を示す一方、試験で不採用とした際の記述はなかった。

　卒業生側は「奨学金の利用を強く勧められたが、不採用なら返済義務があるとの説明はなかった」とした上で、在学中の支給取り消しを定めた規定に反する行為もないと強調。不採用が決まった時期はコロナ禍で、採用人数の抑制といった経営上の事情変更が影響したと疑われるとし、「３人とも成績は良好で、簡易な面接と心理テストだけで不採用と判断するのは不合理。信義則に反する」と訴える。

法人側は取材に「（訴訟について）一切お答えしない」とした。

　被告の女性（３６）は、法人側から「子どもと２人暮らしで、夜勤を安定して行うのが困難」と不採用の理由を示されたとし、「入学前からわかっていたことで納得できない」と話した。

　こうした奨学金は「お礼奉公」とも呼ばれ、看護業界で戦後、看護師不足対策として普及したとされる。病院、学生側双方にメリットがあるが、雇用の流動化が進む中、トラブルが度々起きている。

　全国に看護専門学校は５００校以上あり、看護師向けの転職サイトを運営する「レバレジーズメディカルケア」（東京）によると、ほとんどの大手医療法人は同様の奨学金を導入している。新卒採用者のうち利用者は４～５割を占めるという。

　トラブルで目立つのが、系列病院に就職したものの、厳しい職場環境などになじめずに退職を申し出ると、「奨学金の返済が終わっていない」として慰留されるケースだ。退職すれば、奨学金の一括返済を求められて困る卒業生も少なくない。

　看護師養成制度に詳しい林千冬・神戸市看護大教授は「病院側は学生側にメリットばかりを強調するのではなく、不採用の可能性や奨学金の返済条件など不利益な情報も丁寧に説明し、契約時に書面で合意を得るべきだ。学生側もリスクがあることを十分検討して選択することが大切だろう」と話した。入社初日で辞める「スピード離職」が話題となっている。実際に入社日に退職代行に依頼した新卒の男性を直撃した。

【映像】配属ガチャ？新卒男性が1日で辞めたワケ

　辞めた理由について男性は「4月1日に入社して、配属先発表という時に、自分の希望が全く考慮されていない配属先になっていて、不信感が頂点に達して辞めた。配属ガチャかなと思っていて…」と語った。

　スピード離職の多くの理由は、“配属ガチャ”だという。そんななか、退職代行サービスの利用が急増。退職代行「モームリ」の谷本慎二代表は、「4月なので、新卒の方から入社前と入社後で話が違った、ギャップがあったという声が多い。依頼は初日から、4件ほどあった」と説明した。4月1日から19日までに退職代行サービスに寄せられた依頼は871件、そのうち新社会人は135件。また、東京商工会議所によると、この春の新入社員を対象にした調査で、「定年まで働きたい」と答えたのは21.1パーセント。「チャンスがあれば転職」と答えた割合は26.4パーセントとなり、18年ぶりに定年までを逆転した。

　スピード離職に歯止めをかけたい企業側は新入社員のケアを徹底している。なかには離職予測AIを用いてより精密なケアを実施する企業も。離職予測AIとは、これまでの全社員に関する勤務状況や特性などのデータを入力して、企業ごとの離職者モデルを作成するもの。その精度は脅威の87.3パーセントだという。

　データサイエンス・人工知能の専門家である東京都市大学・白鳥成彦教授は「“上司ガチャ”みたいなものがあって、察しがいい上司、ケアしてくれる上司だったら、ちょっと休みがちだったら、例えば『リフレッシュしてみたらどう？』『休憩してみたらどう？』という形になるが、上司を選ぶとか、なかなかできない」と、上司に不満を感じる新人社員を見つけてケアすることができると解説した。

　この話題に注目した『週刊SPA!』元副編集長の田辺健二氏は、「特にコロナ禍以降、会社への帰属意識がほぼなくなったのと同時に、いわゆるZ世代は、お金よりも承認欲求を優先するので、『定年まで勤めるなんてもうダサい』みたいな考えの人が多い」と前置きした上で「それは間違いではないが、僕らは就職氷河期世代なのでしがみつく人をいっぱい見てきたし、自分も気づいたら20年いた」と述べた。さらに、「そもそもスピード離職しておいて円満を求めるのって、ちょっと話がおかしい。無神経というか、わがまますぎると思う。10年以上働かないと円満って無理なんじゃないかと僕は思う」との見方を示した。

　さらに、依頼が急増している退職代行サービスの利用理由について「リクルーターやいろんな人との向き合いが面倒で気まずいから、『モームリ』に頼むとアンケートで答えている人が多くいた」と語った。山梨・富士河口湖町の「ローソン河口湖駅前店」の屋根越しに見える富士山の撮影などのため、外国人観光客らが殺到し、同町が撮影できないよう黒幕を張る作業を開始したことについて、ローソンの向かいに位置する「井ビシ歯科医院」は1日、公式サイトで、これまでの経緯や事情の説明とともに「景観を損なうもので残念」とする長文の声明を公表した。

  そういえば、繰り返しになりますが、私が嫌いなスポーツは""",
    """今日読んだニュースは以下の通りです。母乳に含まれる抗体が子どもの脳に影響を与える仕組みを、群馬大大学院の定方哲史准教授（４９）と長男で同大医学部３年の瑞樹さん（２０）らの研究グループが解明した。抗体と結合した脳内の細胞が神経細胞の生存に関係するたんぱく質を分泌することを突き止めた。研究成果は国際科学誌「ジャーナル・オブ・ニューロインフラメーション」に発表した。【庄司哲也】

【簡単に解説】母乳が子どもの脳に影響を与える仕組み

　これまでＩｇＧと呼ばれる抗体が母乳から子どもの血液に移送され新生児の免疫力を高めることは分かっていたが、脳への影響は明らかになっていなかった。

　定方准教授らのグループは、実験用の子どものマウスの脳内でＩｇＧが免疫細胞ミクログリアと結合していることに着目。結合したミクログリアが、脳を構成する神経細胞のニューロンの生存に関係するたんぱく質の１型インターフェロンを分泌していることを解明した。

　さらに母親からＩｇＧを受け取るために必要なＦｃＲｎと呼ばれる受容体を欠失させたマウスを調べたところ、ミクログリアや特定のニューロンなど脳内細胞が減少していることも分かった。

　ただ、人間では授乳期に母親が感染症にかかると、ＩｇＧが増えた母乳が、子どもの脳に悪影響を与えてしまう可能性があり、母乳から移送されるＩｇＧの量がどのような影響を与えるかは不明という。

　定方准教授は「ＩｇＧを受け取れないマウスの観察では、社会性行動に有意な変化が見られた。人の母親のＩｇＧが子の脳に良い影響を与えるのか、悪い影響となるのかはさらに研究が必要で、今後明らかにしていきたい」と話している。看護学校生が卒業後に系列病院で一定期間働けば返済が免除される奨学金について、その病院で勤務できない場合、返済義務があるかが大阪地裁の民事訴訟で争われている。学校の運営法人が卒業生３人に返済を求め、卒業生側は法人の都合で不採用にしたのに返済を求めるのは信義則違反だとする。同様の奨学金は看護業界で普及しており、トラブルは後を絶たない。（岸田藍）

　法人は大阪府内の社会医療法人で、精神科病院に併設する看護専門学校（２年課程）を運営している。訴訟記録などによると、学生は入学前の書類審査や面接を経て奨学金を受給でき、卒業後に法人の職員として２年以上勤務すれば返済が全額免除される。

　３人は２０１９年４月に入学した男性１人（２７）、女性２人（３６歳と４４歳）。経済的な理由で、学費名目の奨学金１７８万円をそれぞれ受給した。受給者は同学年約３０人のうち半数ほどだった。

　３人は看護師資格を取得し、卒業後は法人の病院での勤務を希望したが、採用試験で２０年７月に不採用とされ、別の病院に就職した。他の受給者の大半は採用された。

　法人側は「生活状況を踏まえ、奨学金の返済額をそれぞれ１００万円に減らす」と３人に提案したが、合意に至らず、１７８万円ずつの返済を求め、昨年８月に提訴した。

　法人側は、奨学金の貸与が法人職員としての採用を約束するものではないと主張。採用試験で実施した面接や心理テストで、合格水準に達しなかったとし、返済義務があると強調する。

　卒業生側が重視するのは、入学前に配布された奨学金の募集案内の内容だ。受給の条件として「法人の病院に就職する者」を挙げ、貸与取り消しについては「法人への就職を希望しなくなった場合」を示す一方、試験で不採用とした際の記述はなかった。

　卒業生側は「奨学金の利用を強く勧められたが、不採用なら返済義務があるとの説明はなかった」とした上で、在学中の支給取り消しを定めた規定に反する行為もないと強調。不採用が決まった時期はコロナ禍で、採用人数の抑制といった経営上の事情変更が影響したと疑われるとし、「３人とも成績は良好で、簡易な面接と心理テストだけで不採用と判断するのは不合理。信義則に反する」と訴える。

法人側は取材に「（訴訟について）一切お答えしない」とした。

　被告の女性（３６）は、法人側から「子どもと２人暮らしで、夜勤を安定して行うのが困難」と不採用の理由を示されたとし、「入学前からわかっていたことで納得できない」と話した。

　こうした奨学金は「お礼奉公」とも呼ばれ、看護業界で戦後、看護師不足対策として普及したとされる。病院、学生側双方にメリットがあるが、雇用の流動化が進む中、トラブルが度々起きている。

　全国に看護専門学校は５００校以上あり、看護師向けの転職サイトを運営する「レバレジーズメディカルケア」（東京）によると、ほとんどの大手医療法人は同様の奨学金を導入している。新卒採用者のうち利用者は４～５割を占めるという。

　トラブルで目立つのが、系列病院に就職したものの、厳しい職場環境などになじめずに退職を申し出ると、「奨学金の返済が終わっていない」として慰留されるケースだ。退職すれば、奨学金の一括返済を求められて困る卒業生も少なくない。

　看護師養成制度に詳しい林千冬・神戸市看護大教授は「病院側は学生側にメリットばかりを強調するのではなく、不採用の可能性や奨学金の返済条件など不利益な情報も丁寧に説明し、契約時に書面で合意を得るべきだ。学生側もリスクがあることを十分検討して選択することが大切だろう」と話した。入社初日で辞める「スピード離職」が話題となっている。実際に入社日に退職代行に依頼した新卒の男性を直撃した。

【映像】配属ガチャ？新卒男性が1日で辞めたワケ

　辞めた理由について男性は「4月1日に入社して、配属先発表という時に、自分の希望が全く考慮されていない配属先になっていて、不信感が頂点に達して辞めた。配属ガチャかなと思っていて…」と語った。

　スピード離職の多くの理由は、“配属ガチャ”だという。そんななか、退職代行サービスの利用が急増。退職代行「モームリ」の谷本慎二代表は、「4月なので、新卒の方から入社前と入社後で話が違った、ギャップがあったという声が多い。依頼は初日から、4件ほどあった」と説明した。4月1日から19日までに退職代行サービスに寄せられた依頼は871件、そのうち新社会人は135件。また、東京商工会議所によると、この春の新入社員を対象にした調査で、「定年まで働きたい」と答えたのは21.1パーセント。「チャンスがあれば転職」と答えた割合は26.4パーセントとなり、18年ぶりに定年までを逆転した。

　スピード離職に歯止めをかけたい企業側は新入社員のケアを徹底している。なかには離職予測AIを用いてより精密なケアを実施する企業も。離職予測AIとは、これまでの全社員に関する勤務状況や特性などのデータを入力して、企業ごとの離職者モデルを作成するもの。その精度は脅威の87.3パーセントだという。

　データサイエンス・人工知能の専門家である東京都市大学・白鳥成彦教授は「“上司ガチャ”みたいなものがあって、察しがいい上司、ケアしてくれる上司だったら、ちょっと休みがちだったら、例えば『リフレッシュしてみたらどう？』『休憩してみたらどう？』という形になるが、上司を選ぶとか、なかなかできない」と、上司に不満を感じる新人社員を見つけてケアすることができると解説した。

　この話題に注目した『週刊SPA!』元副編集長の田辺健二氏は、「特にコロナ禍以降、会社への帰属意識がほぼなくなったのと同時に、いわゆるZ世代は、お金よりも承認欲求を優先するので、『定年まで勤めるなんてもうダサい』みたいな考えの人が多い」と前置きした上で「それは間違いではないが、僕らは就職氷河期世代なのでしがみつく人をいっぱい見てきたし、自分も気づいたら20年いた」と述べた。さらに、「そもそもスピード離職しておいて円満を求めるのって、ちょっと話がおかしい。無神経というか、わがまますぎると思う。10年以上働かないと円満って無理なんじゃないかと僕は思う」との見方を示した。

　さらに、依頼が急増している退職代行サービスの利用理由について「リクルーターやいろんな人との向き合いが面倒で気まずいから、『モームリ』に頼むとアンケートで答えている人が多くいた」と語った。山梨・富士河口湖町の「ローソン河口湖駅前店」の屋根越しに見える富士山の撮影などのため、外国人観光客らが殺到し、同町が撮影できないよう黒幕を張る作業を開始したことについて、ローソンの向かいに位置する「井ビシ歯科医院」は1日、公式サイトで、これまでの経緯や事情の説明とともに「景観を損なうもので残念」とする長文の声明を公表した。

  私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは""",
    """今日読んだニュースは以下の通りです。母乳に含まれる抗体が子どもの脳に影響を与える仕組みを、群馬大大学院の定方哲史准教授（４９）と長男で同大医学部３年の瑞樹さん（２０）らの研究グループが解明した。抗体と結合した脳内の細胞が神経細胞の生存に関係するたんぱく質を分泌することを突き止めた。研究成果は国際科学誌「ジャーナル・オブ・ニューロインフラメーション」に発表した。【庄司哲也】

【簡単に解説】母乳が子どもの脳に影響を与える仕組み

　これまでＩｇＧと呼ばれる抗体が母乳から子どもの血液に移送され新生児の免疫力を高めることは分かっていたが、脳への影響は明らかになっていなかった。

　定方准教授らのグループは、実験用の子どものマウスの脳内でＩｇＧが免疫細胞ミクログリアと結合していることに着目。結合したミクログリアが、脳を構成する神経細胞のニューロンの生存に関係するたんぱく質の１型インターフェロンを分泌していることを解明した。

　さらに母親からＩｇＧを受け取るために必要なＦｃＲｎと呼ばれる受容体を欠失させたマウスを調べたところ、ミクログリアや特定のニューロンなど脳内細胞が減少していることも分かった。

　ただ、人間では授乳期に母親が感染症にかかると、ＩｇＧが増えた母乳が、子どもの脳に悪影響を与えてしまう可能性があり、母乳から移送されるＩｇＧの量がどのような影響を与えるかは不明という。

　定方准教授は「ＩｇＧを受け取れないマウスの観察では、社会性行動に有意な変化が見られた。人の母親のＩｇＧが子の脳に良い影響を与えるのか、悪い影響となるのかはさらに研究が必要で、今後明らかにしていきたい」と話している。看護学校生が卒業後に系列病院で一定期間働けば返済が免除される奨学金について、その病院で勤務できない場合、返済義務があるかが大阪地裁の民事訴訟で争われている。学校の運営法人が卒業生３人に返済を求め、卒業生側は法人の都合で不採用にしたのに返済を求めるのは信義則違反だとする。同様の奨学金は看護業界で普及しており、トラブルは後を絶たない。（岸田藍）

　法人は大阪府内の社会医療法人で、精神科病院に併設する看護専門学校（２年課程）を運営している。訴訟記録などによると、学生は入学前の書類審査や面接を経て奨学金を受給でき、卒業後に法人の職員として２年以上勤務すれば返済が全額免除される。

　３人は２０１９年４月に入学した男性１人（２７）、女性２人（３６歳と４４歳）。経済的な理由で、学費名目の奨学金１７８万円をそれぞれ受給した。受給者は同学年約３０人のうち半数ほどだった。

　３人は看護師資格を取得し、卒業後は法人の病院での勤務を希望したが、採用試験で２０年７月に不採用とされ、別の病院に就職した。他の受給者の大半は採用された。

　法人側は「生活状況を踏まえ、奨学金の返済額をそれぞれ１００万円に減らす」と３人に提案したが、合意に至らず、１７８万円ずつの返済を求め、昨年８月に提訴した。

　法人側は、奨学金の貸与が法人職員としての採用を約束するものではないと主張。採用試験で実施した面接や心理テストで、合格水準に達しなかったとし、返済義務があると強調する。

　卒業生側が重視するのは、入学前に配布された奨学金の募集案内の内容だ。受給の条件として「法人の病院に就職する者」を挙げ、貸与取り消しについては「法人への就職を希望しなくなった場合」を示す一方、試験で不採用とした際の記述はなかった。

　卒業生側は「奨学金の利用を強く勧められたが、不採用なら返済義務があるとの説明はなかった」とした上で、在学中の支給取り消しを定めた規定に反する行為もないと強調。不採用が決まった時期はコロナ禍で、採用人数の抑制といった経営上の事情変更が影響したと疑われるとし、「３人とも成績は良好で、簡易な面接と心理テストだけで不採用と判断するのは不合理。信義則に反する」と訴える。

法人側は取材に「（訴訟について）一切お答えしない」とした。

　被告の女性（３６）は、法人側から「子どもと２人暮らしで、夜勤を安定して行うのが困難」と不採用の理由を示されたとし、「入学前からわかっていたことで納得できない」と話した。

　こうした奨学金は「お礼奉公」とも呼ばれ、看護業界で戦後、看護師不足対策として普及したとされる。病院、学生側双方にメリットがあるが、雇用の流動化が進む中、トラブルが度々起きている。

　全国に看護専門学校は５００校以上あり、看護師向けの転職サイトを運営する「レバレジーズメディカルケア」（東京）によると、ほとんどの大手医療法人は同様の奨学金を導入している。新卒採用者のうち利用者は４～５割を占めるという。

　トラブルで目立つのが、系列病院に就職したものの、厳しい職場環境などになじめずに退職を申し出ると、「奨学金の返済が終わっていない」として慰留されるケースだ。退職すれば、奨学金の一括返済を求められて困る卒業生も少なくない。

　看護師養成制度に詳しい林千冬・神戸市看護大教授は「病院側は学生側にメリットばかりを強調するのではなく、不採用の可能性や奨学金の返済条件など不利益な情報も丁寧に説明し、契約時に書面で合意を得るべきだ。学生側もリスクがあることを十分検討して選択することが大切だろう」と話した。入社初日で辞める「スピード離職」が話題となっている。実際に入社日に退職代行に依頼した新卒の男性を直撃した。

【映像】配属ガチャ？新卒男性が1日で辞めたワケ

    私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。

　辞めた理由について男性は「4月1日に入社して、配属先発表という時に、自分の希望が全く考慮されていない配属先になっていて、不信感が頂点に達して辞めた。配属ガチャかなと思っていて…」と語った。

　スピード離職の多くの理由は、“配属ガチャ”だという。そんななか、退職代行サービスの利用が急増。退職代行「モームリ」の谷本慎二代表は、「4月なので、新卒の方から入社前と入社後で話が違った、ギャップがあったという声が多い。依頼は初日から、4件ほどあった」と説明した。4月1日から19日までに退職代行サービスに寄せられた依頼は871件、そのうち新社会人は135件。また、東京商工会議所によると、この春の新入社員を対象にした調査で、「定年まで働きたい」と答えたのは21.1パーセント。「チャンスがあれば転職」と答えた割合は26.4パーセントとなり、18年ぶりに定年までを逆転した。

　スピード離職に歯止めをかけたい企業側は新入社員のケアを徹底している。なかには離職予測AIを用いてより精密なケアを実施する企業も。離職予測AIとは、これまでの全社員に関する勤務状況や特性などのデータを入力して、企業ごとの離職者モデルを作成するもの。その精度は脅威の87.3パーセントだという。

　データサイエンス・人工知能の専門家である東京都市大学・白鳥成彦教授は「“上司ガチャ”みたいなものがあって、察しがいい上司、ケアしてくれる上司だったら、ちょっと休みがちだったら、例えば『リフレッシュしてみたらどう？』『休憩してみたらどう？』という形になるが、上司を選ぶとか、なかなかできない」と、上司に不満を感じる新人社員を見つけてケアすることができると解説した。

　この話題に注目した『週刊SPA!』元副編集長の田辺健二氏は、「特にコロナ禍以降、会社への帰属意識がほぼなくなったのと同時に、いわゆるZ世代は、お金よりも承認欲求を優先するので、『定年まで勤めるなんてもうダサい』みたいな考えの人が多い」と前置きした上で「それは間違いではないが、僕らは就職氷河期世代なのでしがみつく人をいっぱい見てきたし、自分も気づいたら20年いた」と述べた。さらに、「そもそもスピード離職しておいて円満を求めるのって、ちょっと話がおかしい。無神経というか、わがまますぎると思う。10年以上働かないと円満って無理なんじゃないかと僕は思う」との見方を示した。

　さらに、依頼が急増している退職代行サービスの利用理由について「リクルーターやいろんな人との向き合いが面倒で気まずいから、『モームリ』に頼むとアンケートで答えている人が多くいた」と語った。山梨・富士河口湖町の「ローソン河口湖駅前店」の屋根越しに見える富士山の撮影などのため、外国人観光客らが殺到し、同町が撮影できないよう黒幕を張る作業を開始したことについて、ローソンの向かいに位置する「井ビシ歯科医院」は1日、公式サイトで、これまでの経緯や事情の説明とともに「景観を損なうもので残念」とする長文の声明を公表した。

  そういえば、繰り返しになりますが、私が嫌いなスポーツは""",
]

In [8]:
for t in texts:
    generated, wps = inference_retnet(model_retnet, tok_retnet, t, mode="recurrent")


# of input tokens: 205 tokens
Reccurent Inference: 2.3498284816741943 seconds
Input: 私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。今日読んだニュースは以下の通りです。「米国の中央銀行にあたる米連邦準備制度理事会（FRB）は、1日に開いた米連邦公開市場委員会（FOMC）で、6会合連続で政策金利を据え置くと決めた。現状維持は市場の予想通りで、FRBのパウエル議長は記者会見で、高金利を長く保つ意向を明らかにした一方で、再び利上げする可能性は「低いと思う」と述べた。
パウエル氏が利上げの可能性は低いと言及したことで、米金利の上昇による日米の金利差拡大への観測が後退。ニューヨーク市場で、円相場はやや円高ドル安方向に振れ、米東部時間1日午後3時半（日本時間2日午前4時半）ごろには1ドル=157円台半ばで取引されていた。」そういえば、繰り返しになりますが、私が嫌いなスポーツは
Output: 野球と卓球です。私が嫌いなスポーツは野球と卓球です。
「米国の中央銀行にあたる米連邦準備制度理事会(FRB)は、6会合連続で政策金利を据え置くと決めた。現状維持は市場の予想通り
Throughput (recurrent): 37.44954182243216 wps

# of input tokens: 46 tokens
Reccurent Inference: 2.234332323074341 seconds
Input: 私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。私が嫌いなスポーツは野球と卓球です。そういえば、繰り返しになりますが、私が嫌いなスポーツは
Output: 野球と卓球です。
私は、野球と卓球は、野球と卓球は、野球と卓球は、野球と卓球は、野球と卓球は、野球と卓球は、野球と卓球は、野球と卓球は、野球と
Throughput (recurrent): 31.776830718855283 wps

# of input tokens: 11 tokens
Reccurent Inference: 2.2381753921508

In [12]:
# model = RetNetForCausalLM.from_pretrained(
#     "/nas/k_ishikawa/results/RetNet/retnet3b_ctxLen2048_anonym/checkpoint-280",
#     device_map="auto",
# )
model = RetNetForCausalLM.from_pretrained(
    "/nas/k_ishikawa/results/RetNet/retnet3b_anonym_3ep/checkpoint-860",
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  4.00s/it]


In [13]:
tok = AutoTokenizer.from_pretrained("cyberagent/calm2-7b-chat")

In [18]:
text = """
USER: アキラです。好きな食べ物は焼売です。
ASSISTANT: マコトです。嫌いなスポーツはカバディです。
USER: ノベルティソング、イエイ！
ASSISTANT: ちょっと耳なじみがあまりないかと思います。一口に言うと、日本でいうコミックソングのことに当たるんですけど、ただコミックソングに限定されるものではないですね。<|endoftext|>
USER: 原義はね、大瀧詠一さんがよく使うんで調べたんですけど、「珍しさ」「目新しさ」っていう意味。日本で言ったら「珍奇な」という感じですかね。
ASSISTANT: なぜ珍奇なのかということを、24時台はしていきたいなと思っております。日本のロックの中でノベルティ性のある音楽。これをスージーさんが説明してください。<|endoftext|>
USER: まずは音階です。ブルーノートっていうふうに専門用語で言うんですけど、日本に入ってきたのは割とノベルティ、コミック、ちょっとおもしろげな曲からそういう音が入ってきたという話をしたいと思います。まず1曲目、ザ・スパイダースですね。1967年にこんなおしゃれでブルージーな音楽を作っておりました。
（音源再生：『恋のドクター』）
ASSISTANT: かっこいいね。<|endoftext|>
USER: 堺正章さんのシャウトがいいですよ。
（♪「注射ピュー」）<|endoftext|>
USER: 『恋のドクター』っていうザ・スパイダースの曲ですけど、ブルーノートを多用して、全編ブルーノートみたいなメロディーなんですね。（♪ギター演奏）<|endoftext|>
USER: この……
ASSISTANT: 「♪ティーン」のところですね。<|endoftext|>
USER: 普通だったらこういう音は使わないんですけれども。作詞作曲かまやつひろし。スパイダースというおしゃれで先進的なロックバンドが、ブルーノートを大胆に使った。歌詞が恋だの愛だのとかいうまじめな歌じゃなくて、『恋のドクター』。「注射ピュー」（笑）。
新しい音楽をコミカルに取り入れた、スパイダースのすごさ
ASSISTANT: これ1967年ですか。<|endoftext|>
USER: 早いですよ。
ASSISTANT: 67年の時点でこんなに黒人音楽っぽい音を聴いた当時のリスナーたちはどう思ったんですかね。<|endoftext|>
USER: 当時タイガース、テンプターズというのが人気バンドで、その前はブルーコメッツとスパイダース。ブルーコメッツはレコード大賞取った『ブルー・シャトウ』とか、割とまじめ。
ASSISTANT: 「♪森と泉に」ですね。<|endoftext|>
USER: 対してスパイダースはショーアップされてて、かつエンターテイメント性に溢れている。ブルーノートとか新しい西洋音楽の手法を、愛だの恋だのはなくて「注射ピュー」みたいな諧謔性、ノベルティ性の中で導入していった。親しみやすい歌詞で、音楽はすごくソリッドで先進的というのがスパイダースの方法論。
ASSISTANT: スパイダースは良家の子女の集まり、言ってみれば。<|endoftext|>
USER: 東京のお坊ちゃんですね。
ASSISTANT: 遊びを知ってる人たちが、音楽を楽し気にやってるっていう感じも素敵だったんでしょうね。<|endoftext|>
USER: 最新の洋楽をそのまま提示するんじゃなくおもしろさとか、1億人に分かるような形でコーティングして出す。そこで日本のブルーノートの歴史が始まった。
ASSISTANT: なるほどね。<|endoftext|>
USER: 急にロックンロールが広まったわけじゃなくて、ずっとノベルティの世界でジワジワとブルーノートが浸透していきます。次のこの曲は全然ロックンロールではないんですけど、一番はじめのリフにすごく濃いブルーノートが。
（音源再生：『ピンポンパン体操』）
ASSISTANT: わあ、これ懐かしい。<|endoftext|>
USER: 1971年、フジテレビで当時大ヒット。
ASSISTANT: これ冒頭のとこでしょう、ブルーノート。
（♪ギター演奏）
ASSISTANT: ピンポンパポンの「ピン」ね。この怪しげな響きがなんともブルーノート。<|endoftext|>
USER: ここで出てくるのが、作曲家の小林亜星。
子供向けの曲にブルーノートをしのばせ、世に広めた小林亜星
ASSISTANT: 出た、「パッ！とさいでりあ」（笑）。<|endoftext|>
USER: 日本のノベルティ界のナンバーワン作曲家。作詞阿久悠、作曲小林亜星。
ASSISTANT: すごいコンビだ。<|endoftext|>
USER: 「♪すきすきすきすき　アッコちゃーん　アッコちゃん　すきすき」も小林亜星さん。
ASSISTANT: すごいね。ブルーノートとかロックンロール的なニュアンスを、小林亜星さんという作家を通して、子ども向けに下してくれてたわけですね。<|endoftext|>
USER: ダイレクトに大人に行くんじゃなく、子どもを経由して日本に広まっていったということですね。
ASSISTANT: 子どもは頭が柔らかいから。固定観念なくして、そういうところでちゃんと啓蒙するという。<|endoftext|>
USER: この曲はメロディーもたくさんあって偏執的な音で、小林亜星がめっちゃ遊んでる。一説によると、260万枚くらい売れた。童謡チャートにカウントされたんで、『およげ！たいやきくん』とかと比較はされないけど大ヒットしてる。当時の阿久悠に言わせると、キャバレーで女の子がこれでちょっとエッチな体操したくらい、世の中で広まった。
ASSISTANT: 夜の盛り場とブルーノートっていうことで線が引けるわけですね。<|endoftext|>
USER: 60年代だと『伊勢佐木町ブルース』とかね。
ASSISTANT: あれもブルーノートになってるんですか。<|endoftext|>
USER: 「♪ドゥドゥビドゥビドゥビドゥビドゥバ～」。
ASSISTANT: あそこか。<|endoftext|>
USER: ミ♭ですね。いかがわしいエロブルーノート。
ASSISTANT: 半音階っていうのが当時の日本人が聴いたときに、笑いというか気持ちがざわざわしたんでしょうね。<|endoftext|>
USER: エロとかコミカルとか、普通のヒットソングにはない感情を揺り動かすツールとしてブルーノートが使われた。スパイダース『恋のドクター』があって、矢継ぎ早に小林亜星がアニメとかCM、ピンポンパン体操「♪ズンズンズンズンピンポン」でわさわさと気持ちを揺れさせて。
ASSISTANT: このあたりが日本におけるブルーノート元年。<|endoftext|>
USER: 昔、渋谷陽一さんが言ってましたけれども、「僕はロックンロールっていうのをビートルズから学びました」とかウソだと。私的に言えば、日本人は、ロックンロールは『ピンポンパン体操』で学んだ（笑）。
ASSISTANT: すごいですね。<|endoftext|>
USER: そして1972年キャロルのデビューですね。キャロルはブルーノート使ってますけど、あまりノベルティ性は高くない。割とまじめにやってた。スパイダースとキャロルと同じフィリップスレーベルで、73年にデビューしたのがフィンガー5。これは子ども向けのロックロールっていうことで大ヒット。
次に日本人の隅の隅までブルーノートとロックンロールが浸透したんじゃないという、歴史のエポックとなる大ヒットしたこの曲。
（音源再生：『スモーキン・ブギ』）
ロックンロールを日本の隅々に浸透させた、ダウン・タウン・ブギウギ・バンド
<|endoftext|>
USER: ダウン・タウン・ブギウギ・バンド、『スモーキン・ブギ』。
ASSISTANT: たまらないですよ。僕ももちろん子どもだったんでブルーノートかどうかは分からないけど、でも全編その世界っていうのはたぶん認識してました。<|endoftext|>
USER: イントロから、ロックンロールの一番代表的なリフですけれど。
ASSISTANT: 小気味よくて、子ども心にもおかしなことを歌ってるのも分かるし、耳が気持ちいいっていう感触は覚えてますよ。<|endoftext|>
USER: やっぱりこのフレーズですね。「目覚めの一プク　食後の一プク　授業をサボって　喫茶店で一プク」。リズム感を出すために日本語で詰まる音、小っちゃい「っ」を多用することによって口の端に乗って気持ちいいリズムを作ってる。あと一回聴いたら忘れない、「スー パッパッ」。歌詞は不良少年がタバコを吸ってる、高校生くらいの舞台。シリアスなこと語ってるんじゃなくて、非常にコミカル。
ロックンロールというイディオムを日本に導入するときに、スパイダース、小林亜星、キャロルも少し、そしてフィンガー5、ダウン・タウン・ブギウギ・バンド。子どもも含めてちょっとコミカルで、ノベルティな形で日本でロックンロールというものが浸透してきたんじゃないかという歴史ですね。
ASSISTANT: 不良性の一つのアイコンとして、都市部にキャロルという存在があった。のパロディみたいな世界にダウン・タウン・ブギウギ・バンド。その時点でノベルティ性があります。<|endoftext|>
USER: ダウン・タウン・ブギウギ・バンドって名前自体がもうノベルティがある。ここから、よりノベルティ度を強くした横浜銀蝿につながっていく。日本におけるロックンロールは洋楽じゃなく、こういうところから浸透してきたんじゃないか。もっと言うと、今かけたのは全部高卒ロック。はっぴいえんどとかシュガー・ベイブとか佐野元春みたいな、イメージとしての大学卒が聴いてそうな音楽じゃなく、こういう高卒ロック、小学ロックみたいな。
ASSISTANT: 町場のリアルですよ。
ASSISTANT: そういうところで受け入れられていく経過はあったと思います、絶対ね。<|endoftext|>
USER: ついつい日本語ロックの始祖ははっぴいえんどという話になってくるけど、『ピンポンパン体操』も大事なんじゃないかと。
ASSISTANT: そうですよ。僕なんかの小学生だって分かるんだもん。おもしろいっていうキャッチの仕方があったわけで、ノベルティソングってそういう威力がある。<|endoftext|>
USER: そういえば、全然話は変わるけど、嫌いなスポーツは何？
ASSISTANT: """

In [19]:
inputs = tok(text, return_tensors="pt").to(model.device)

In [20]:
outputs = model.custom_generate(
    **inputs, parallel_compute_prompt=True, max_new_tokens=100
)

In [21]:
print(tok.decode(outputs[0], skip_special_tokens=True))


USER: アキラです。好きな食べ物は焼売です。
ASSISTANT: マコトです。嫌いなスポーツはカバディです。
USER: ノベルティソング、イエイ！
ASSISTANT: ちょっと耳なじみがあまりないかと思います。一口に言うと、日本でいうコミックソングのことに当たるんですけど、ただコミックソングに限定されるものではないですね。
USER: 原義はね、大瀧詠一さんがよく使うんで調べたんですけど、「珍しさ」「目新しさ」っていう意味。日本で言ったら「珍奇な」という感じですかね。
ASSISTANT: なぜ珍奇なのかということを、24時台はしていきたいなと思っております。日本のロックの中でノベルティ性のある音楽。これをスージーさんが説明してください。
USER: まずは音階です。ブルーノートっていうふうに専門用語で言うんですけど、日本に入ってきたのは割とノベルティ、コミック、ちょっとおもしろげな曲からそういう音が入ってきたという話をしたいと思います。まず1曲目、ザ・スパイダースですね。1967年にこんなおしゃれでブルージーな音楽を作っておりました。
（音源再生：『恋のドクター』）
ASSISTANT: かっこいいね。
USER: 堺正章さんのシャウトがいいですよ。
（♪「注射ピュー」）
USER: 『恋のドクター』っていうザ・スパイダースの曲ですけど、ブルーノートを多用して、全編ブルーノートみたいなメロディーなんですね。（♪ギター演奏）
USER: この……
ASSISTANT: 「♪ティーン」のところですね。
USER: 普通だったらこういう音は使わないんですけれども。作詞作曲かまやつひろし。スパイダースというおしゃれで先進的なロックバンドが、ブルーノートを大胆に使った。歌詞が恋だの愛だのとかいうまじめな歌じゃなくて、『恋のドクター』。「注射ピュー」（笑）。
新しい音楽をコミカルに取り入れた、スパイダースのすごさ
ASSISTANT: これ1967年ですか。
USER: 早いですよ。
ASSISTANT: 67年の時点でこんなに黒人音楽っぽい音を聴いた当時のリスナーたちはどう思ったんですかね。
USER: 当時タイガース、テンプターズというのが人気バンドで、その前はブルーコメッツとスパイダース。ブルーコメッツはレコード大賞取った『ブルー・シャトウ』とか、割とまじめ

In [13]:
inputs.input_ids.shape

torch.Size([1, 2013])